In [ ]:
#################INSTALL iopath, fvcore, optuna diocanegg###########
### Cose da fare: 
# impostare il resize bene, per ora è a 28.
#  Impostare is_rgb per avere 1 o 3 canali. 
# Impostare una evaluation corretta


import os
from datetime import datetime
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import random_split, Dataset, DataLoader
from PIL import Image
from fvcore.nn import FlopCountAnalysis
import optuna
from optuna.trial import TrialState
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


/home/gabriele/miniconda3/envs/galaxy_project/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/gabriele/miniconda3/envs/galaxy_project/lib/python3.10/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647352509/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


### TODO
- Fine tuning dei parametri ---> optuna
- Capire come salvare i dati [(documentazione)](https://docs.pytorch.org/tutorials/beginner/saving_loading_models.html) FATTO credo
- mapping delle probabilità da aggiungere alla rete. 

In [2]:
class GalaxyJungle(Dataset): # sarebbe interessante implementare un rescale/crop
    
    #the init function initializes the directory containing the image,
    #the annotations file,
    #and both transforms
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None, is_rgb=False):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.is_rgb = is_rgb

    #returns number of samples in the dataset
    def __len__(self):
        return len(self.img_labels)

    #loads a sample from the dataset
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, str(self.img_labels.iloc[idx, 0])) + '.jpg'
        #retrieves the image
        image = Image.open(img_path)
        if not self.is_rgb: image = image.convert('L')
        #retrieves corresponding label
        label = self.img_labels.iloc[idx, 1:]
        #if possible, transform the image and the label into a tensor.
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(label.values, dtype=torch.float32)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label#, self.img_labels.iloc[idx, 0]
    

transform = transforms.Compose([
    transforms.ToTensor(), #fa già la normalizzazione se l'immagine non è un tensore
    # sarebbe interessante implementare un random crop prima del center crop per decentrare un poco le immagini????
    transforms.RandomHorizontalFlip(), # horizontal flip
    transforms.RandomVerticalFlip(), 
    transforms.Resize((28,28))# vertical flip
    #transforms.CenterCrop(csize)          #CROP
    ]) #transforms.compose per fare una pipe di transformazioni



DS = GalaxyJungle('../data/training/training_solutions_rev1.csv', '../data/training/', transform=transform,is_rgb=False)
training, test_set = random_split(DS, [.8, .2])
#training[0]


img, lab, indx = DS.__getitem__(0)
#print(lab)
#print(img)         #3D TENSOR    
if is_rgb:
    fig, ax = plt.subplots(1,3, figsize=(24,7))
    color = ['Reds', 'Greens', 'Blues']
    for i,j in enumerate(img):
        ax[i].imshow(j, cmap=color[i])
else:
    fig, ax = plt.subplots(1,1, figsize=(24,7))
    ax.imshow(img[0], cmap='magma')
#print(img.shape)


## Neural Net

In [3]:
class GalaxyNet(nn.Module):
    def __init__(self, trial, num_conv_layers, num_filters, num_neurons, drop_conv2, drop_fc1):
        
        super(GalaxyNet, self).__init__()
        input_size = 28
        kernel_size = 3
        output_size=input_size
        padding = 0
        dilation = 1
        stride = 2

        self.convs = nn.ModuleList([nn.Conv2d(1, num_filters[0], kernel_size=(3,3))])
        output_size = output_size - kernel_size + 1
        output_size = output_size // 2
        #output_size = int(((input_size + 2 * padding - dilation * (kernel_size -1)) / stride) + 1 )
        for i in range(1, num_conv_layers):
            self.convs.append(nn.Conv2d(in_channels=num_filters[i-1], out_channels=num_filters[i], kernel_size=(3,3)))
            output_size = output_size - kernel_size + 1
            output_size = output_size // 2
            #output_size = int(((input_size + 2 * padding - dilation * (kernel_size -1)) / stride) + 1 )
        self.conv2_drop = nn.Dropout2d(p=drop_conv2)
        self.out_feature = num_filters[num_conv_layers-1] * output_size * output_size
        self.fc1 = nn.Linear(self.out_feature, num_neurons) # fc è fully connected, #120 neuroni che prendono l'output
        self.fc2 = nn.Linear(num_neurons, 37)# un altro fc layer che prende dai 120- neuroni e connette a 84 neuroni
        self.p1 = drop_fc1
        #i numeri non vincolati sono il primo 6 e il primo 16 e poi i numeri di neuroni
        ## Instance variables:
        for i in range(1, num_conv_layers):
            nn.init.kaiming_normal_(self.convs[i].weight, nonlinearity='relu')
            if self.convs[i].bias is not None:
                nn.init.constant_(self.convs[i].bias,0)
        nn.init.kaiming_normal_(self.fc1.weight, nonlinearity='relu')

    def forward(self, x):
        for i, conv_i in enumerate(self.convs):  # For each convolutional layer
            if i == 2:  # Add dropout if layer 2
                x = F.relu(F.max_pool2d(self.conv2_drop(conv_i(x)), 2))  # Conv_i, dropout, max-pooling, RelU
            else:
                x = F.relu(F.max_pool2d(conv_i(x), 2))                   # Conv_i, max-pooling, RelU

        x = x.view(x.size(0),-1)
        #x = x.flatten()                    # Flatten tensor
        x = F.relu(self.fc1(x))                              # FC1, RelU
        x = F.dropout(x, p=self.p1, training=self.training)  # Apply dropout after FC1 only when training
        x = self.fc2(x)    
        return x
    
    #def convool_size(input_dimension, padding, dilation, kernel_size, stride):
        '''
        Returns Conv2d and Max_pool output size for a square input tensor.
        '''
        return ((input_dimension + 2 * padding - dilation * (kernel_size - 1) - 1) / stride) + 1
    
    #def log_the_loss(self, item,epoch=False, validation=False): # per avere una history della loss???
   #     if epoch and not validation:
   #         self.loss_dict['epoch'].append(item)
   #     elif not epoch and not validation:
   #         self.loss_dict['batch'].append(item)
   #     elif validation and epoch:
   #         self.loss_dict['vepoch'].append(item)
   #     elif validation and not epoch:
   #         self.loss_dict['vbatch'].append(item)
   #     return item

In [4]:
def train(network, optimizer):
    network.train()
    for batch_index, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data.to(device))
        loss = F.mse_loss(output, target.to(device))
        loss.backward()
        optimizer.step()

def test(network):
    
    network.eval()
    total_loss = 0
    with torch.no_grad():
        for  data,target in test_loader:
            output = network(data.to(device))
            loss = F.mse_loss(output, target.to(device), reduction='sum')
            total_loss += loss.item()
    avg_loss = total_loss /len(test_loader.dataset)

    return avg_loss

In [10]:
def objective(trial):
    num_conv_layers = trial.suggest_int("num_conv_layers", 2, 3)
    num_filters = [int(trial.suggest_discrete_uniform("num_filters_"+str(i), 16, 128, 16)) for i in range(num_conv_layers)]
    num_neurons = trial.suggest_int("num_neurons",10,400,10)
    drop_conv2 = trial.suggest_float("drop_conv2", 0.2, 0.5)
    drop_fc1 = trial.suggest_float("drop_fc1", 0.2,0.5)

    model = GalaxyNet(trial, num_conv_layers, num_filters, num_neurons, drop_conv2, drop_fc1).to(device)
    #model = GalaxyNet(trial, 2, num_filters, 10, 0.1, 0.1).to(device)
    
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    for epoch in range(nepochs):
        train(model,optimizer)
        loss = test(model)
        trial.report(loss, epoch)
    return loss

In [11]:
random_seed = 42
torch.manual_seed(random_seed)

nepochs = 1                         # Number of training epochs
batch_size_train = 64                # Batch size for training data
batch_size_test = 420               # Batch size for testing data
number_of_trials = 2                # Number of Optuna trials

train_loader = DataLoader(training,batch_size=batch_size_train, shuffle=True, num_workers=8)
test_loader = DataLoader(test_set, batch_size=batch_size_test, shuffle=True, num_workers=8)

study = optuna.create_study(direction='minimize') 
study.optimize(objective, n_trials=number_of_trials)
best_params = study.best_params


[I 2025-05-15 12:29:47,028] A new study created in memory with name: no-name-649e4dca-5059-42eb-8b1d-8a3a394985d2
/tmp/ipykernel_34479/478379788.py:3: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  num_filters = [int(trial.suggest_discrete_uniform("num_filters_"+str(i), 16, 128, 16)) for i in range(num_conv_layers)]
/tmp/ipykernel_34479/478379788.py:4: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  num_neurons = trial.suggest_int("num_neurons",10,400,10)
[I 2025-05-15 12:30:17,172] Trial 0 finished with value: 1.0023457522901442 and parameters: {'num_conv_layers': 2, 'num_filters_0': 48.0, 'num_filters_1': 96.0, 'num_neurons': 260, 'drop_conv2': 0.279551308543565, 'drop_fc1': 0.23874116706577286, 'optimizer': 'Adam', 'lr': 0.062050700999970086}. 

## Traininig + validation

In [8]:
def one_epoch_train(verbose=False):
    running_loss = 0.
    last_loss = 0.
    gnet.train(True)
    for i, data in enumerate(train_loader):
        inputs, labels, idx = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = gnet(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        RMSEloss =  np.sqrt(loss.item())
        if verbose and i%10==0: print(f'Batch {i+1}/{len(train_loader)} - Loss: {RMSEloss:.3f}')
        running_loss += RMSEloss
        gnet.log_the_loss(RMSEloss, epoch=False, validation=False)
        if i == len(train_loader)-1:
            epochmean_loss = running_loss / len(train_loader)
            if verbose: print(f"---\nEpoch {epoch} - Loss: {epochmean_loss:.3f}\n---")
            gnet.log_the_loss(epochmean_loss, epoch=True, validation=False)
            last_loss = RMSEloss
            if verbose: print(f"---\nEpoch {epoch} - Loss: {last_loss:.3f}\n---")
    return last_loss

def one_epoch_eval(verbose=False):
    gnet.eval()
    running_validation_loss = 0.
    if verbose: print('Evaluation...')
    with torch.no_grad():
        for i, vdata in enumerate(test_loader):
            inputs, labels, idx = vdata
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = gnet(inputs)
            loss = loss_function(outputs, labels)
            RMSEloss =  np.sqrt(loss.item())
            running_validation_loss += RMSEloss
            gnet.log_the_loss(RMSEloss,epoch=False, validation = True)
    mean_vloss = gnet.log_the_loss(RMSEloss/(i+1), epoch=True, validation=True)
    if verbose: print(f"---\nValidation Loss: {mean_vloss:.3f}\n---")
    return mean_vloss
    

In [ ]:
for epoch in range(0,2):
    print(f'Training epoch {epoch}')
    epoch_last_loss = one_epoch_train(verbose=True)
    #print(f'Epoch {epoch} - Loss: {epoch_last_loss:.3f}')
    epoch_vloss = one_epoch_eval(verbose=True)
    #print(f'Epoch {epoch} - Validation Loss: {epoch_vloss:.3f}')

Training epoch 0
Batch 1/385 - Loss: 0.275
Batch 11/385 - Loss: 0.283
Batch 21/385 - Loss: 0.266
Batch 31/385 - Loss: 0.276
Batch 41/385 - Loss: 0.264
Batch 51/385 - Loss: 0.266
Batch 61/385 - Loss: 0.255
Batch 71/385 - Loss: 0.255
Batch 81/385 - Loss: 0.247
Batch 91/385 - Loss: 0.247
Batch 101/385 - Loss: 0.244
Batch 111/385 - Loss: 0.242
Batch 121/385 - Loss: 0.235
Batch 131/385 - Loss: 0.226
Batch 141/385 - Loss: 0.227
Batch 151/385 - Loss: 0.231
Batch 161/385 - Loss: 0.212
Batch 171/385 - Loss: 0.213
Batch 181/385 - Loss: 0.210
Batch 191/385 - Loss: 0.214
Batch 201/385 - Loss: 0.201
Batch 211/385 - Loss: 0.193
Batch 221/385 - Loss: 0.188
Batch 231/385 - Loss: 0.186
Batch 241/385 - Loss: 0.187
Batch 251/385 - Loss: 0.177
Batch 261/385 - Loss: 0.178
Batch 271/385 - Loss: 0.180
Batch 281/385 - Loss: 0.178
Batch 291/385 - Loss: 0.176
Batch 301/385 - Loss: 0.172
Batch 311/385 - Loss: 0.157
Batch 321/385 - Loss: 0.184
Batch 331/385 - Loss: 0.162
Batch 341/385 - Loss: 0.171
Batch 351/385 

## Saving the model

In [ ]:
#explore our model parameter so we don't have to retrain if we want to use it again
now = datetime.now().strftime("%d-%m_%H-%M-%S")
model_last_name = f'gnet/trained_gnet_{now}.pth'
torch.save({
    'model_state_dict' : gnet.state_dict(), # tutti i pesi
    'optimizer_state_dict' : optimizer.state_dict(), # values of the optimizer, loss is just the last loss value.
    'loss' : gnet.loss_dict,
    'batchsize_train_test' : (batch_size_train, batch_size_test),
    'device' : device,
    'is_rgb' : is_rgb,
    'crop_size' : csize,
    'kernel_size' : kernel_size,
    'out_channels' : out_channels,
    'feature_map_2' : feature_map_2,
    'max_pool_kernel' : max_pool_kernel,
    'loss_function' : str(loss_function),
    'optimizer' : str(optimizer),
    'epoch' : len(gnet.loss_dict['epoch'])}, model_last_name)

## Loading the model

In [ ]:
gnet = GalaxyNet()
gnet.load_state_dict(torch.load(model_last_name)['model_state_dict'])
optimizer.load_state_dict(torch.load(model_last_name)['optimizer_state_dict'])
loss = torch.load(model_last_name)['loss']

/tmp/ipykernel_26239/2436948299.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  gnet.load_state_dict(torch.load(model_last_name)['model_state_dict'])
/tmp/ipykernel_2623